# Testing Functional Connectivities
This script is for me to experiment/test generating functional connectivities from the data.

In [ ]:
from nilearn import plotting, image
from os import listdir
from os.path import isfile, join
import numpy as np

In [ ]:
subject_path = '/imaging3/owenlab/wilson/MovieData/Release8/Age10/BioBank_Analysis_All/aamod_smooth_00001/sub-NDARAP912JK3'

files = [join(subject_path, f) for f in listdir(subject_path) if isfile(join(subject_path, f))]
print(files[1])

In [ ]:
concat_file = image.concat_imgs(files[1])

In [ ]:
selected_volumes = image.index_img(concat_file, slice(3, 5))

for img in image.iter_img(selected_volumes):
    plotting.view_img(img)

In [ ]:
plotting.plot_glass_brain(image.mean_img(concat_file), threshold=None, display_mode='lzr')

In [ ]:
from nilearn import datasets

dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

print('Atlas ROIs are located in nifti image (4D) at: %s' %
      atlas_filename)  # 4D data

In [ ]:
from nilearn import datasets
from nilearn.input_data import NiftiSpheresMasker

power = datasets.fetch_coords_power_2011()
coords = np.vstack((power.rois['x'], power.rois['y'], power.rois['z'])).T

spheres_masker = NiftiSpheresMasker(
    seeds=coords, smoothing_fwhm=6, radius=5.,
    detrend=True, standardize=True, low_pass=0.1, high_pass=0.01, t_r=2)

time_series = spheres_masker.fit_transform(concat_file)
print('Time series has {0} samples'.format(time_series.shape[0]))

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', verbose=1)

# Here we go from nifti files to the signal time series in a numpy
# array. Note how we give confounds to be regressed out during signal
# extraction
time_series = masker.fit_transform(concat_file)

In [ ]:
from nilearn.connectome import ConnectivityMeasure
correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([time_series])[0]

# Plot the correlation matrix
from nilearn import plotting

# Make a large figure
# Mask the main diagonal for visualization:
np.fill_diagonal(correlation_matrix, 0)
# The labels we have start with the background (0), hence we skip the
# first label
# matrices are ordered for block-like representation
# plotting.plot_matrix(correlation_matrix, figure=(10, 8), labels=labels[1:],
#                      vmax=0.8, vmin=-0.8, reorder=True)

plotting.plot_matrix(correlation_matrix, vmin=-1., vmax=1., colorbar=True, figure=(20, 20),
                     title='Power correlation matrix')

# Tweak edge_threshold to keep only the strongest connections.
plotting.plot_connectome(correlation_matrix, coords, title='Power correlation graph',
                         edge_threshold='99.5%', node_size=20, colorbar=True)